In [4]:
import pandas as pd
import requests
import time
import json
import datetime
import timedelta

In [ ]:
weather_df = pd.read_csv("./jupyter/2024_forecast.csv")

In [70]:
del weather_df['Unnamed: 0']

In [71]:
weather_df.head(1)

,month,day,forecast,rain,low_temp_low,low_temp_high,high_temp_low,high_temp_high
0,1,1,transition,23%,High Temperature Forecast 35,45.0,Low Temperature Forecast 15,25.0


In [72]:
weather_df['date'] = ""

In [136]:
start_date = '2024-1-2'
end_date = '2024-1-2'
params = '{"elems":[{"name":"maxt","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_100"},{"reduce":"pct_ge_90"},{"reduce":"pct_ge_50"},{"reduce":"pct_le_32"}]},{"name":"mint","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_70"},{"reduce":"pct_ge_50"},{"reduce":"pct_le_32"},{"reduce":"pct_le_0"}]},{"name":"pcpn","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_0.01"},{"reduce":"pct_ge_0.10"},{"reduce":"pct_ge_0.50"},{"reduce":"pct_ge_1.0"}]},{"name":"snow","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_0.1"},{"reduce":"pct_ge_1.0"},{"reduce":"pct_ge_3.0"},{"reduce":"pct_ge_6.0"}]},{"name":"snwd","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_1"},{"reduce":"pct_ge_3"},{"reduce":"pct_ge_6"},{"reduce":"pct_ge_12"}]}],"sid":"TOPthr 9","sDate":"{start_date}","eDate":"{end_date}"}'
params = params.replace('{start_date}', start_date)
params = params.replace('{end_date}', end_date)
param_obj = json.loads(params)
url = 'https://data.rcc-acis.org/StnData'
req = requests.post(url, data={'params': params, 'output': 'json'})

In [137]:
date, max_temp, min_temp, precip, snow, snow_depth = req.json()['data'][0]
day_data = {'date': date, 'actual_max_temp': max_temp, "actual_min_temp": min_temp,
            "actual_precip": precip, "actual_snow": snow, "actual_snow_depth": snow_depth}
ephemera_url = "http://ira.local:5000/api/weather/add"
requests.post(ephemera_url, json=day_data).text

In [75]:
actuals_df = pd.DataFrame([day_data])

In [132]:
actuals_df

,date,actual_max_temp,actual_min_temp,actual_precip,actual_snow,actual_snow_depth
0,2024-01-01,39,24,0.00,M,M


In [133]:
for i, r in weather_df.iterrows():
    current_date = f"2024-{r['month']:02d}-{r['day']:02d}"
    weather_df.loc[i, 'date'] = current_date

In [134]:
day_df_json = pd.merge(weather_df, actuals_df,
                       how='inner').to_json(orient='records')
pd.merge(weather_df, actuals_df,
         how='inner')

,month,day,forecast,rain,low_temp_low,low_temp_high,high_temp_low,high_temp_high,date,actual_max_temp,actual_min_temp,actual_precip,actual_snow,actual_snow_depth
0,1,1,transition,23%,High Temperature Forecast 35,45.0,Low Temperature Forecast 15,25.0,2024-01-01,39,24,0.00,M,M


In [5]:
ephemera_url = "http://ira.local:5000/api/weather"
requests.get(ephemera_url).json()

[{'_id': {'$oid': '6594f80192594cb87c609eb8'},
  'date': '2024-01-01',
  'actual_max_temp': '39',
  'actual_min_temp': '24',
  'actual_precip': '0.00',
  'actual_snow': 'M',
  'actual_snow_depth': 'M'},
 {'_id': {'$oid': '659575453eb39eb2035ad8f8'},
  'date': '2024-01-02',
  'actual_max_temp': '44',
  'actual_min_temp': '21',
  'actual_precip': '0.00',
  'actual_snow': '0.0',
  'actual_snow_depth': '0'}]

In [9]:
start_date = datetime.strftime(
    datetime.now() - timedelta(1), '%Y-%m-%d')  # '2024-1-2'
end_date = datetime.strftime(
    datetime.now() - timedelta(1), '%Y-%m-%d')  # '2024-1-2'
params = '{"elems":[{"name":"maxt","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_100"},{"reduce":"pct_ge_90"},{"reduce":"pct_ge_50"},{"reduce":"pct_le_32"}]},{"name":"mint","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_70"},{"reduce":"pct_ge_50"},{"reduce":"pct_le_32"},{"reduce":"pct_le_0"}]},{"name":"pcpn","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_0.01"},{"reduce":"pct_ge_0.10"},{"reduce":"pct_ge_0.50"},{"reduce":"pct_ge_1.0"}]},{"name":"snow","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_0.1"},{"reduce":"pct_ge_1.0"},{"reduce":"pct_ge_3.0"},{"reduce":"pct_ge_6.0"}]},{"name":"snwd","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_1"},{"reduce":"pct_ge_3"},{"reduce":"pct_ge_6"},{"reduce":"pct_ge_12"}]}],"sid":"TOPthr 9","sDate":"{start_date}","eDate":"{end_date}"}'
params = params.replace('{start_date}', start_date)
params = params.replace('{end_date}', end_date)
param_obj = json.loads(params)
url = 'https://data.rcc-acis.org/StnData'
req = requests.post(url, data={'params': params, 'output': 'json'})

date, max_temp, min_temp, precip, snow, snow_depth = req.json()['data'][0]
day_data = {'date': date, 'actual_max_temp': max_temp, "actual_min_temp": min_temp,
            "actual_precip": precip, "actual_snow": snow, "actual_snow_depth": snow_depth}

{'date': '2024-01-02',
 'actual_max_temp': '44',
 'actual_min_temp': '21',
 'actual_precip': '0.00',
 'actual_snow': 'M',
 'actual_snow_depth': 'M'}

In [8]:
ephemera_url = "http://ira.local:5000/api/weather/add"
requests.post(ephemera_url, json=day_data).text

'2024-01-02'